In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests
## Michael Triner 9 November 2018

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [8]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

In [9]:
lamML = 1/np.average(my_data)
testStat = 2*((20*log(lamML)-lamML*np.sum(my_data))-(20*log(3)-3*np.sum(my_data)))
testStat

4.719222360188457

In [27]:
pVal = 1-stats.chi2.cdf(testStat,1)
pVal

0.029827229194775207

P-Value is < 0.05 thus we reject the null hypothesis

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [55]:
critVal = stats.chi2.ppf(.95,1)
critVal

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [72]:
testStats =[]
for i in np.arange(10000):
    my_data = stats.expon.rvs(size=20,scale=1/5)
    lamML = 1/np.average(my_data)
    testStat = 2*((20*log(lamML)-lamML*np.sum(my_data))-(20*log(3)-3*np.sum(my_data)))
    testStats = np.append(testStats,testStat)
reject = np.count_nonzero(testStats>=critVal)/10000
print('Reject the null: ',reject*100, '% of the time')

Reject the null:  59.35 % of the time


Repeat for a sample size of 50. What do you expect to happen to power? 

In [73]:
testStats =[]
for i in np.arange(10000):
    my_data = stats.expon.rvs(size=50,scale=1/5)
    lamML = 1/np.average(my_data)
    testStat = 2*((50*log(lamML)-lamML*np.sum(my_data))-(50*log(3)-3*np.sum(my_data)))
    testStats = np.append(testStats,testStat)
reject = np.count_nonzero(testStats>=critVal)/10000
print('Reject the null: ',reject*100, '% of the time')

Reject the null:  95.04 % of the time


### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [78]:
Sampledata = stats.expon.rvs(size=20,scale=1/5)
SampletestStat = np.average(Sampledata)
SampletestStat

0.1629730175431169

In [80]:
simStat = []
for i in np.arange(10000):
    simData = stats.expon.rvs(size=20,scale=1/3)
    statistic = np.average(simData)
    simStat = np.append(simStat,statistic)

np.count_nonzero(simStat>=SampletestStat)/10000

0.9962

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [2]:
hyp_true_data = []
for i in np.arange(10000):
    ts = 1/np.average(stats.expon.rvs(scale =1/3, size =20))
    hyp_true_data = np.append(hyp_true_data, ts)

In [6]:
upperReject = 4.5
lowerReject = 2.17
print('Reject interval: (',lowerReject,',',upperReject,')')

Reject interval: ( 2.17 , 4.5 )


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [11]:
n = 20
ML_lams = []
reject = []
for i in np.arange(10000):
    ts = 1/(np.average(np.random.choice(my_data, n, replace =True)))
    ML_lams = np.append(ML_lams, ts)
    if (upperReject < ts) or (lowerReject > ts):
        reject = np.append(reject, True)
    else:
        reject = np.append(reject, False)
sum(reject)/10000

0.773

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [13]:
hyptrue = []
for i in np.arange(10000):
    ts = 1/np.average(stats.expon.rvs(scale =1/3, size =50))
    hyptrue = np.append(hyptrue, ts)

In [14]:
upperReject50 = 3.86
lowerReject50 = 2.43
print('Reject interval: (',lowerReject50,',',upperReject50,')')

Reject interval: ( 2.43 , 3.86 )


In [15]:
n = 50
ML_lams = []
reject = []
for i in np.arange(10000):
    ts = 1/(np.average(np.random.choice(my_data, n, replace =True)))
    ML_lams = np.append(ML_lams, ts)
    if (upperReject50 < ts) or (lowerReject50 > ts):
        reject = np.append(reject, True)
    else:
        reject = np.append(reject, False)
sum(reject)/10000

0.9944